## P0 Scanner
#### Original Author: Emil Sekerinski, revised March 2020
The scanner reads the characters of the source consecutively and recognizes symbols they form:
- procedure `init(src)` initializes the scanner
- procedure `getSym()` recognizes the next symbol and assigns it to variables `sym` and `val`.
- procedure `mark(msg)` prints an error message at the current location in the source.

Symbols are encoded by integer constants.

In [ ]:
TIMES = 1; DIV = 2; MOD = 3; AND = 4; PLUS = 5; MINUS = 6
OR = 7; EQ = 8; NE = 9; LT = 10; GT = 11; LE = 12; GE = 13
PERIOD = 14; COMMA = 15; COLON = 16; NOT = 17; LPAREN = 18
RPAREN = 19; LBRAK = 20; RBRAK = 21; LARROW = 22; RARROW = 23
OF = 24; THEN = 24; DO = 26; BECOMES = 27; NUMBER = 28; IDENT = 29
SEMICOLON = 30; ELSE = 31; IF = 32; WHILE = 33; ARRAY = 34
RECORD = 35; CONST = 36; TYPE = 37; VAR = 38; PROCEDURE = 39
PROGRAM = 40; INDENT = 41; DEDENT = 42; NL = 43; EOF = 44;
MALLOC = 45

Following variables determine the state of the scanner:

- `(line, pos)` is the location of the current symbol in source
- `(lastline, lastpos)` is used to more accurately report errors
- `ch` is the current character
- `sym` the current symbol
- if `sym` is `NUMBER`, `val` is the value of the number
- if `sym` is `IDENT`, `val` is the identifier string
- `source` is the string with the source program

The source is specified as a parameter to the procedure `init`:

In [ ]:
def init(src):
    global line, lastline, pos, lastpos
    global ch, sym, val, source, index, indents
    line, lastline = 0, 1
    pos, lastpos = 1, 1
    ch, sym, val, source, index = '\n', None, None, src, 0
    indents = [1]; getChar(); getSym()

Procedure `getChar()` assigns the next character in `ch`, or assigns `chr(0)` at the end of the source. Variables `line`, `pos` are updated with the current location in the source and `lastline`, `lastpos` are updated with the location of the previously read character.

In [ ]:
def getChar():
    global line, lastline, pos, lastpos, ch, index
    if index == len(source): ch = chr(0); pos = 1
    else:
        lastpos = pos
        if ch == '\n':
            pos, line = 1, line + 1
        else:
            lastline, pos = line, pos + 1
        ch, index = source[index], index + 1

Procedure `mark(msg)` prints an error message with the current location in the source. To avoid a cascade of errors, only one error message at a source location is printed.

In [ ]:
def mark(msg):
    raise Exception('line ' + str(lastline) + ' pos ' + str(lastpos) + ' ' + msg)

Procedure `number()` parses

    number ::= digit {digit}
    digit ::= '0' | ... | '9'

If the number fits in 32 bits, sets `sym` to `NUMBER` and assigns to number to `val`, otherwise reports an error.

In [ ]:
def number():
    global sym, val
    sym, val = NUMBER, 0
    while '0' <= ch <= '9':
        val = 10 * val + int(ch)
        getChar()
    if val >= 2**31:
        mark('number too large'); val = 0

Procedure `identKW()` parses

    identKW ::= identifier - keyword | keyword
    identifier ::= letter {letter | digit}
    letter ::= 'A' | ... | 'Z' | 'a' | ... | 'z'
    keyword ::= 'div' | 'mod' | 'and' | 'or' | 'of' | 'then' | 'do' | 'not' | 'end' | 'else' | 'if' | 'while' |
                         'array' | 'record' | 'const' | 'type' | 'var' | 'procedure' | 'begin' | 'program'

The longest sequence of character that matches `letter {letter | digit}` is read. If that sequence is a keyword, `sym` is set accordingly, otherwise `sym` is set to `IDENT`.

In [ ]:
KEYWORDS = \
    {'div': DIV, 'mod': MOD, 'and': AND, 'or': OR, 'of': OF, 'then': THEN,
    'do': DO, 'else': ELSE, 'if': IF, 'while': WHILE, 'array': ARRAY,
    'record': RECORD, 'const': CONST, 'type': TYPE, 'var': VAR,
    'procedure': PROCEDURE, 'program': PROGRAM, 'malloc': MALLOC}

def identKW():
    global sym, val
    start = index - 1
    while ('A' <= ch <= 'Z') or ('a' <= ch <= 'z') or \
          ('0' <= ch <= '9'): getChar()
    val = source[start:index-1]
    sym = KEYWORDS[val] if val in KEYWORDS else IDENT

Procedure `comment()` parses

    comment ::= '{' {character} '}'
    
If a comment is not terminated, an error is reported, otherwise the comment is skipped.

In [ ]:
def comment():
    while chr(0) != ch != '}': getChar()
    if ch == chr(0): mark('comment not terminated')
    else: getChar()

Procedure `getSym()` parses

    symbol ::= { ' ' } ( { '\n' {' '} } | identKW | number | comment | '×' | '+' | '-' | '=' | '≠' |
                        '<' | '≤' | '>' | '≥' | ';' | ',' | ':' | ':=' | '.' | '¬' | '(' |  ')' | '[' | ']' )

If a valid symbol is recognized, `sym` is set accordingly, otherwise an error is reported. The longest match is used for recognizing operators. Blanks that are not at the beginning of a line are skipped. A stack, `indents`, is used to keep track if blanks at the beginning of a line are either ignored or recognized as `INDENT` or `DEDENT`. On the first symbol of a line, `newline` is set to `True` if the indentation is the same as that of the previous line; for all subsequent symbols, `newline` is set to `False`. At the end of the source, `sym` is set to `EOF`.

In [ ]:
def getSym():
    global sym, indents, newline
    if pos < indents[0]:
        indents = indents[1:]; sym = DEDENT
    else:
        while ch == ' ': getChar() # skip blanks between symbols
        if ch == '\n': # possibly INDENT, DEDENT
            while ch == '\n': # skip blank lines
                getChar()
                while ch == ' ': getChar() # skip indentation
            if pos < indents[0]: indents = indents[1:]; sym = DEDENT; return
            elif pos > indents[0]: sym, indents = INDENT, [pos] + indents; return
        newline = pos == indents[0]
        if 'A' <= ch <= 'Z' or 'a' <= ch <= 'z': identKW()
        elif '0' <= ch <= '9': number()
        elif ch == '{': comment(); getSym()
        elif ch == '×': getChar(); sym = TIMES
        elif ch == '+': getChar(); sym = PLUS
        elif ch == '-': getChar(); sym = MINUS
        elif ch == '=': getChar(); sym = EQ
        elif ch == '≠': getChar(); sym = NE
        elif ch == '<': getChar(); sym = LT
        elif ch == '≤': getChar(); sym = LE
        elif ch == '>': getChar(); sym = GT
        elif ch == '≥': getChar(); sym = GE
        elif ch == ';': getChar(); sym = SEMICOLON
        elif ch == ',': getChar(); sym = COMMA
        elif ch == ':':
            getChar()
            if ch == '=': getChar(); sym = BECOMES
            else: sym = COLON
        elif ch == '.': getChar(); sym = PERIOD
        elif ch == '¬': getChar(); sym = NOT
        elif ch == '(': getChar(); sym = LPAREN
        elif ch == ')': getChar(); sym = RPAREN
        elif ch == '[': getChar(); sym = LBRAK
        elif ch == ']': getChar(); sym = RBRAK
        elif ch == '←': getChar(); sym = LARROW
        elif ch == '→': getChar(); sym = RARROW
        elif ch == chr(0): sym = EOF
        else: mark('illegal character')

Procedure `getSym()` parses

    symbol ::= { ' ' } ( '\n' {' '} | identKW | number | comment | '×' | '+' | '-' | '=' | '≠' |
                        '<' | '≤' | '>' | '≥' | ';' | ',' | ':' | ':=' | '.' | '¬' | '(' |  ')' | '[' | ']' )

If a valid symbol is recognized, `sym` is set accordingly, otherwise an error is reported. The longest match is used for recognizing operators. Blanks that are not at the beginning of a line are skipped. A stack, `indents`, is used to keep track if blanks at the beginning of a line are recognized as either `NL`, `INDENT`, or `DEDENT`. At the end of the source, `sym` is set to `EOF`.

In [ ]:
# def getSym():
#     global sym, indents
#     if pos < indents[0]:
#         indents = indents[1:]; sym = NL if pos == indents[0] else DEDENT
#     else:
#         while ch == ' ': getChar() # skip blanks between symbols
#         if ch == '\n': # NL, INDENT, (multiple) DEDENT (followed by NL)
#             getChar()
#             while ch == ' ': getChar() # skip indentation
#             if pos < indents[0]: sym = DEDENT
#             elif pos > indents[0]: sym, indents = INDENT, [pos] + indents
#             else: sym = NL
#         elif 'A' <= ch <= 'Z' or 'a' <= ch <= 'z': identKW()
#         elif '0' <= ch <= '9': number()
#         elif ch == '{': comment(); getSym()
#         elif ch == '×': getChar(); sym = TIMES
#         elif ch == '+': getChar(); sym = PLUS
#         elif ch == '-': getChar(); sym = MINUS
#         elif ch == '=': getChar(); sym = EQ
#         elif ch == '≠': getChar(); sym = NE
#         elif ch == '<': getChar(); sym = LT
#         elif ch == '≤': getChar(); sym = LE
#         elif ch == '>': getChar(); sym = GT
#         elif ch == '≥': getChar(); sym = GE
#         elif ch == ';': getChar(); sym = SEMICOLON
#         elif ch == ',': getChar(); sym = COMMA
#         elif ch == ':':
#             getChar()
#             if ch == '=': getChar(); sym = BECOMES
#             else: sym = COLON
#         elif ch == '.': getChar(); sym = PERIOD
#         elif ch == '¬': getChar(); sym = NOT
#         elif ch == '(': getChar(); sym = LPAREN
#         elif ch == ')': getChar(); sym = RPAREN
#         elif ch == '[': getChar(); sym = LBRAK
#         elif ch == ']': getChar(); sym = RBRAK
#         elif ch == '←': getChar(); sym = LARROW
#         elif ch == '→': getChar(); sym = RARROW
#         elif ch == chr(0): sym = EOF
#         else: mark('illegal character')